## IDEA <br>

- Use OMDB API: http://www.omdbapi.com/#usage
- Do the API request with the search, movies with word 'love', extract info from x pages
- Like that get the list of titles of the movies
- Do another API request title by title to get more info about the movies
- Put info from both requests to one df
- Scrap IMDB or Wikipedia based on title or IMDB ID to get info from there
- Check if some data matching: Release year, Genre, Director, IMDB Rating
- Could also get more data about the director from Wiki or IMDB

## API <br>

- For the API portion of the project will need to make calls to your chosen API, successfully obtain a response, request data, convert it into a Pandas data frame, and export it as a CSV file.

In [1]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re


#check if using this library
import time

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

- Doing the search of the titles that include word 'love' in x pages

- converting names into the correct format to later pass them to the URL

In [4]:
#generating several jsons from different pages
#in the range indicate how many pages want to check (API in general gives MAX movies per search)
#and then adding all the movie names to one list
#per each loop generating json for one page and extracting titles

def formatted_titles_api(url_pat):
    final_list = []
    for page in range(1,5):
        page = str(page)
        url = url_pat + page
        response = requests.get(url)
        results = response.json()
        title_box = results.get('Search')
        title_list = [title.get('Title') for title in title_box]
        final_list += title_list
        titles_formatted = [string.replace(" ", "+") for string in final_list]
    return titles_formatted
        

In [5]:
#calling the function

url_pat= 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie&page='
titles_formatted = formatted_titles_api(url_pat)

- Getting the info about each movie from the list

In [6]:
def movies_info_api(url_pattern):
    appended_dfs=[]
    for title in titles_formatted:
        url = url_pattern + title
        response = requests.get(url)
        results = response.json()
        flattened_data = json_normalize(results)
        #appending dataframes to the list to later concatenate them to one dataframe
        appended_dfs.append(flattened_data)
        love_movies = pd.concat(appended_dfs, ignore_index = True) #ignoring index, to have 0,1,2...n, if not, get 0,0,0,...

    return love_movies

In [7]:
url_pattern = 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t='
love_movies = movies_info_api(url_pattern)
love_movies.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,"Crazy, Stupid, Love.",2011,PG-13,29 Jul 2011,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Emma Stone","A middle-aged husband's life changes dramatically when his wife asks him for a divorce. He seeks to rediscover his manhood with the help of a newfound friend, Jacob, learning to pick up girls at bars.",English,USA,Nominated for 1 Golden Globe. Another 3 wins & 22 nominations.,https://m.media-amazon.com/images/M/MV5BMTg2MjkwMTM0NF5BMl5BanBnXkFtZTcwMzc4NDg2NQ@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '79%'}, {'Source': 'Metacritic', 'Value': '68/100'}]",68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True
1,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,PG,29 Jan 1964,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (screenplay), Peter George (screenplay), Peter George (based on the book: ""Red Alert"" by)","Peter Sellers, George C. Scott, Sterling Hayden, Keenan Wynn",An insane general triggers a path to nuclear holocaust that a War Room full of politicians and generals frantically tries to stop.,"English, Russian","UK, USA",Nominated for 4 Oscars. Another 13 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BZWI3ZTMxNjctMjdlNS00NmUwLWFiM2YtZDUyY2I3N2MxYTE0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '98%'}, {'Source': 'Metacritic', 'Value': '97/100'}]",97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True
2,"Love, Actually",2003,R,14 Nov 2003,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Colin Firth","Follows the lives of eight very different couples in dealing with their love lives in various loosely interrelated tales all set during a frantic month before Christmas in London, England.","English, Portuguese, French","UK, USA, France",Nominated for 2 Golden Globes. Another 10 wins & 27 nominations.,https://m.media-amazon.com/images/M/MV5BMTY4NjQ5NDc0Nl5BMl5BanBnXkFtZTYwNjk5NDM3._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '55/100'}]",55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True
3,Shakespeare in Love,1998,R,08 Jan 1999,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell, Tim McMullan","The world's greatest ever playwright, William Shakespeare, is young, out of ideas and short of cash, but meets his ideal woman and is inspired to write one of his most famous plays.",English,"USA, UK",Won 7 Oscars. Another 57 wins & 87 nominations.,https://m.media-amazon.com/images/M/MV5BM2ZkNjM5MjEtNTBlMC00OTI5LTgyYmEtZDljMzNmNzhiNzY0XkEyXkFqcGdeQXVyNDYyMDk5MTU@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}, {'Source': 'Metacritic', 'Value': '87/100'}]",87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True
4,P.S. I Love You,2007,PG-13,21 Dec 2007,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Rogers (screenplay), Cecelia Ahern (novel)","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina Gershon",A young widow discovers that her late husband has left her 10 messages intended to help ease her pain and start a new life.,English,USA,2 wins & 4 nominations.,https://m.media-amazon.com/images/M/MV5BNTg2MDg4MjI5NV5BMl5BanBnXkFtZTcwMzQ0MDczMw@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Data

## WEB SCRAPING <br>

- For the web scraping portion of the project, you will need to scrape the HTML from your chosen page, parse the HTML to extract the necessary information, and either save the results to a text (txt) file if it is text or into a CSV file if it is tabular data.

In [8]:
#taking movie IDs from API df, to scrape them in IMDB

id_list = list(love_movies['imdbID'])

In [9]:
#creating function to scrape data from IMDB

def scraped_info_to_dataframe(url_pattern):
    info_final = []

    for m_id in id_list:
        url = url_pattern + m_id

        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')

        #extracting director
        summary = soup.find_all('div', {'class':'credit_summary_item'})
        director = summary[0].get_text().strip().split('\n')[1]

        #extracting title
        title_box = soup.find_all('h1', {'class':''})
        for t in title_box:
            title = t.text.split('\xa0')[0]
        #if title is in Spanish, thenoriginal title class exists, then take the title from there, to have it in English    
        original_title_box = soup.find_all('div', {'class':'originalTitle'})
        if original_title_box:
            title_long = [t.text.split('\xa0')[0] for t in original_title_box]
            pattern = re.compile("(.*?)\s*\((.*?)\)") #only taking the string part before the ()
            for string in title_long:
                title = pattern.match(string).group(1)

        #extracting rating
        rating_box = soup.find_all('div', {'class':'ratingValue'})
        rates = [rate.text.strip() for rate in rating_box]
        pattern = '\d+\D\d' #extracting format only x.x
        for ratio in rates:
            rating = re.findall(pattern,ratio)[0]

        #extracting year
        year_box = soup.find_all('h1', {'class':''})
        year_long = [y.text.strip().split('\xa0')[1] for y in year_box]
        pattern = re.compile(".*?\((.*?)\)") #extrcting year from ()
        for string in year_long:
            year = pattern.match(string).group(1)
        
        #adding links to videos per movie
        videos = 'https://www.imdb.com/title/x/videogallery/?ref_=tt_ql_pv_2'
        for m_id in id_list:
            videos = videos.replace('x', m_id)
        
        #adding all info to one nested list
        info_final += [[title, year, rating, director, videos]]

    #creating df with the scraped info
    colnames = ['Title', 'Year', 'imdbRating','Director', 'VideoLink']
    love_movies_imdb = pd.DataFrame(info_final)
    love_movies_imdb.columns = colnames
    
    return love_movies_imdb


In [10]:
#calling the function

url_pattern = 'https://www.imdb.com/title/'

love_movies_imdb = scraped_info_to_dataframe(url_pattern)
love_movies_imdb.head()

,Title,Year,imdbRating,Director,VideoLink
0,"Crazy, Stupid, Love.",2011,7.4,"Glenn Ficarra, John Requa",https://www.imdb.com/title/tt1570728/videogallery/?ref_=tt_ql_pv_2
1,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,8.4,Stanley Kubrick,https://www.imdb.com/title/tt1570728/videogallery/?ref_=tt_ql_pv_2
2,Love Actually,2003,7.6,Richard Curtis,https://www.imdb.com/title/tt1570728/videogallery/?ref_=tt_ql_pv_2
3,Shakespeare in Love,1998,7.1,John Madden,https://www.imdb.com/title/tt1570728/videogallery/?ref_=tt_ql_pv_2
4,P.S. I Love You,2007,7.0,Richard LaGravenese,https://www.imdb.com/title/tt1570728/videogallery/?ref_=tt_ql_pv_2


## DATA CHECK <br>

- Check if data scraped from IMDB mathes with the data from API

## RATING COMPARISON <br>

- See if there is relation in higher ratings when the release date is closer to the valentines day
- Calculate days until Valentines day

In [11]:
love_movies.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,"Crazy, Stupid, Love.",2011,PG-13,29 Jul 2011,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Emma Stone","A middle-aged husband's life changes dramatically when his wife asks him for a divorce. He seeks to rediscover his manhood with the help of a newfound friend, Jacob, learning to pick up girls at bars.",English,USA,Nominated for 1 Golden Globe. Another 3 wins & 22 nominations.,https://m.media-amazon.com/images/M/MV5BMTg2MjkwMTM0NF5BMl5BanBnXkFtZTcwMzc4NDg2NQ@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '79%'}, {'Source': 'Metacritic', 'Value': '68/100'}]",68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True
1,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,PG,29 Jan 1964,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (screenplay), Peter George (screenplay), Peter George (based on the book: ""Red Alert"" by)","Peter Sellers, George C. Scott, Sterling Hayden, Keenan Wynn",An insane general triggers a path to nuclear holocaust that a War Room full of politicians and generals frantically tries to stop.,"English, Russian","UK, USA",Nominated for 4 Oscars. Another 13 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BZWI3ZTMxNjctMjdlNS00NmUwLWFiM2YtZDUyY2I3N2MxYTE0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '98%'}, {'Source': 'Metacritic', 'Value': '97/100'}]",97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True
2,"Love, Actually",2003,R,14 Nov 2003,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Colin Firth","Follows the lives of eight very different couples in dealing with their love lives in various loosely interrelated tales all set during a frantic month before Christmas in London, England.","English, Portuguese, French","UK, USA, France",Nominated for 2 Golden Globes. Another 10 wins & 27 nominations.,https://m.media-amazon.com/images/M/MV5BMTY4NjQ5NDc0Nl5BMl5BanBnXkFtZTYwNjk5NDM3._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '55/100'}]",55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True
3,Shakespeare in Love,1998,R,08 Jan 1999,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell, Tim McMullan","The world's greatest ever playwright, William Shakespeare, is young, out of ideas and short of cash, but meets his ideal woman and is inspired to write one of his most famous plays.",English,"USA, UK",Won 7 Oscars. Another 57 wins & 87 nominations.,https://m.media-amazon.com/images/M/MV5BM2ZkNjM5MjEtNTBlMC00OTI5LTgyYmEtZDljMzNmNzhiNzY0XkEyXkFqcGdeQXVyNDYyMDk5MTU@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}, {'Source': 'Metacritic', 'Value': '87/100'}]",87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True
4,P.S. I Love You,2007,PG-13,21 Dec 2007,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Rogers (screenplay), Cecelia Ahern (novel)","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina Gershon",A young widow discovers that her late husband has left her 10 messages intended to help ease her pain and start a new life.,English,USA,2 wins & 4 nominations.,https://m.media-amazon.com/images/M/MV5BNTg2MDg4MjI5NV5BMl5BanBnXkFtZTcwMzQ0MDczMw@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Data

In [12]:
love_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 25 columns):
Title         40 non-null object
Year          40 non-null object
Rated         40 non-null object
Released      40 non-null object
Runtime       40 non-null object
Genre         40 non-null object
Director      40 non-null object
Writer        40 non-null object
Actors        40 non-null object
Plot          40 non-null object
Language      40 non-null object
Country       40 non-null object
Awards        40 non-null object
Poster        40 non-null object
Ratings       40 non-null object
Metascore     40 non-null object
imdbRating    40 non-null object
imdbVotes     40 non-null object
imdbID        40 non-null object
Type          40 non-null object
DVD           40 non-null object
BoxOffice     40 non-null object
Production    40 non-null object
Website       40 non-null object
Response      40 non-null object
dtypes: object(25)
memory usage: 7.9+ KB


In [13]:
love_movies['Released']= pd.to_datetime(love_movies['Released']) 

In [14]:
love_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 25 columns):
Title         40 non-null object
Year          40 non-null object
Rated         40 non-null object
Released      40 non-null datetime64[ns]
Runtime       40 non-null object
Genre         40 non-null object
Director      40 non-null object
Writer        40 non-null object
Actors        40 non-null object
Plot          40 non-null object
Language      40 non-null object
Country       40 non-null object
Awards        40 non-null object
Poster        40 non-null object
Ratings       40 non-null object
Metascore     40 non-null object
imdbRating    40 non-null object
imdbVotes     40 non-null object
imdbID        40 non-null object
Type          40 non-null object
DVD           40 non-null object
BoxOffice     40 non-null object
Production    40 non-null object
Website       40 non-null object
Response      40 non-null object
dtypes: datetime64[ns](1), object(24)
memory usage: 7.9+ KB


In [15]:
love_movies.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,"Crazy, Stupid, Love.",2011,PG-13,2011-07-29,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Emma Stone","A middle-aged husband's life changes dramatically when his wife asks him for a divorce. He seeks to rediscover his manhood with the help of a newfound friend, Jacob, learning to pick up girls at bars.",English,USA,Nominated for 1 Golden Globe. Another 3 wins & 22 nominations.,https://m.media-amazon.com/images/M/MV5BMTg2MjkwMTM0NF5BMl5BanBnXkFtZTcwMzc4NDg2NQ@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '79%'}, {'Source': 'Metacritic', 'Value': '68/100'}]",68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True
1,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,PG,1964-01-29,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (screenplay), Peter George (screenplay), Peter George (based on the book: ""Red Alert"" by)","Peter Sellers, George C. Scott, Sterling Hayden, Keenan Wynn",An insane general triggers a path to nuclear holocaust that a War Room full of politicians and generals frantically tries to stop.,"English, Russian","UK, USA",Nominated for 4 Oscars. Another 13 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BZWI3ZTMxNjctMjdlNS00NmUwLWFiM2YtZDUyY2I3N2MxYTE0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '98%'}, {'Source': 'Metacritic', 'Value': '97/100'}]",97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True
2,"Love, Actually",2003,R,2003-11-14,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Colin Firth","Follows the lives of eight very different couples in dealing with their love lives in various loosely interrelated tales all set during a frantic month before Christmas in London, England.","English, Portuguese, French","UK, USA, France",Nominated for 2 Golden Globes. Another 10 wins & 27 nominations.,https://m.media-amazon.com/images/M/MV5BMTY4NjQ5NDc0Nl5BMl5BanBnXkFtZTYwNjk5NDM3._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '55/100'}]",55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True
3,Shakespeare in Love,1998,R,1999-01-08,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell, Tim McMullan","The world's greatest ever playwright, William Shakespeare, is young, out of ideas and short of cash, but meets his ideal woman and is inspired to write one of his most famous plays.",English,"USA, UK",Won 7 Oscars. Another 57 wins & 87 nominations.,https://m.media-amazon.com/images/M/MV5BM2ZkNjM5MjEtNTBlMC00OTI5LTgyYmEtZDljMzNmNzhiNzY0XkEyXkFqcGdeQXVyNDYyMDk5MTU@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}, {'Source': 'Metacritic', 'Value': '87/100'}]",87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True
4,P.S. I Love You,2007,PG-13,2007-12-21,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Rogers (screenplay), Cecelia Ahern (novel)","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina Gershon",A young widow discovers that her late husband has left her 10 messages intended to help ease her pain and start a new life.,English,USA,2 wins & 4 nominations.,https://m.media-amazon.com/images/M/MV5BNTg2MDg4MjI5NV5BMl5BanBnXkFtZTcwMzQ0MDczMw@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database'

In [38]:
#still needs string

#from datetime import datetime as dt

#love_movies['Released']  = love_movies['Released'].apply(lambda s: dt.strptime(s, "%Y-%m-%d"))
#love_movies['NewDate1'] = love_movies['Released'].dt.strftime("%m-%d")


TypeError: strptime() argument 1 must be str, not Timestamp

In [41]:
#don't want to convert to string

love_movies['ReleasedMonth'] = love_movies['Released'].dt.month
love_movies['ReleasedDay'] = love_movies['Released'].dt.day

#love_movies['ReleasedMonth'] = love_movies['ReleasedMonth'].astype('str')
#love_movies['ReleasedDay'] = love_movies['ReleasedDay'].astype('str')


#love_movies['MonthDay'] = love_movies['ReleasedMonth'] + '-' + love_movies['ReleasedDay']

#love_movies['MonthsToValentine'] = love_movies['ReleasedMonth']

In [40]:
love_movies.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,ReleasedMonth,ReleasedDay,MonthDay,MonthsToValentine
0,"Crazy, Stupid, Love.",2011,PG-13,2011-07-29,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Emma Stone","A middle-aged husband's life changes dramatically when his wife asks him for a divorce. He seeks to rediscover his manhood with the help of a newfound friend, Jacob, learning to pick up girls at bars.",English,USA,Nominated for 1 Golden Globe. Another 3 wins & 22 nominations.,https://m.media-amazon.com/images/M/MV5BMTg2MjkwMTM0NF5BMl5BanBnXkFtZTcwMzc4NDg2NQ@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '79%'}, {'Source': 'Metacritic', 'Value': '68/100'}]",68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True,7,29,7-29,5
1,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,PG,1964-01-29,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (screenplay), Peter George (screenplay), Peter George (based on the book: ""Red Alert"" by)","Peter Sellers, George C. Scott, Sterling Hayden, Keenan Wynn",An insane general triggers a path to nuclear holocaust that a War Room full of politicians and generals frantically tries to stop.,"English, Russian","UK, USA",Nominated for 4 Oscars. Another 13 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BZWI3ZTMxNjctMjdlNS00NmUwLWFiM2YtZDUyY2I3N2MxYTE0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '98%'}, {'Source': 'Metacritic', 'Value': '97/100'}]",97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True,1,29,1-29,-1
2,"Love, Actually",2003,R,2003-11-14,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Colin Firth","Follows the lives of eight very different couples in dealing with their love lives in various loosely interrelated tales all set during a frantic month before Christmas in London, England.","English, Portuguese, French","UK, USA, France",Nominated for 2 Golden Globes. Another 10 wins & 27 nominations.,https://m.media-amazon.com/images/M/MV5BMTY4NjQ5NDc0Nl5BMl5BanBnXkFtZTYwNjk5NDM3._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '55/100'}]",55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True,11,14,11-14,9
3,Shakespeare in Love,1998,R,1999-01-08,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell, Tim McMullan","The world's greatest ever playwright, William Shakespeare, is young, out of ideas and short of cash, but meets his ideal woman and is inspired to write one of his most famous plays.",English,"USA, UK",Won 7 Oscars. Another 57 wins & 87 nominations.,https://m.media-amazon.com/images/M/MV5BM2ZkNjM5MjEtNTBlMC00OTI5LTgyYmEtZDljMzNmNzhiNzY0XkEyXkFqcGdeQXVyNDYyMDk5MTU@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}, {'Source': 'Metacritic', 'Value': '87/100'}]",87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True,1,8,1-8,-1
4,P.S. I Love You,2007,PG-13,2007-12-21,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Rogers (screenplay), Cecelia Ahern (novel)","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina Gershon",A young widow discovers that her late husband has left her 10 messages intended to help ease her pain and start a new life.,English,USA,2 wins & 4 nominations.,https://m.media-amazon.com/images/

In [ ]:
#later take year from year column

love_movies['ValentinesDay_no_year'] = '-02-14'

In [ ]:
love_movies['ValentinesDay'] = love_movies['Year'] + love_movies['ValentinesDay_no_year']


In [ ]:
love_movies=love_movies.drop(['ValentinesDay_no_year'],axis=1)

In [ ]:
love_movies.head()

In [ ]:
love_movies['ValentinesDay']= pd.to_datetime(love_movies['ValentinesDay']) 

In [ ]:
love_movies.info()

In [ ]:
love_movies['DaysToValentine'] = love_movies['Released'] - love_movies['ValentinesDay']

In [ ]:
love_movies['DaysToValentine'] = love_movies['DaysToValentine'].astype('str')

In [ ]:
love_movies['DaysToValentine'] = love_movies['DaysToValentine'].str.extract(r'(\d+)')

In [ ]:
love_movies.head()

In [ ]:
love_movies['imdbRating'] = love_movies['imdbRating'].astype('float')

In [ ]:
love_movies['DaysToValentine'] = love_movies['DaysToValentine'].astype('int')

In [ ]:
#doesn't really say anything, could put to buckets

love_movies.plot(kind='bar',x='DaysToValentine',y='imdbRating')


In [ ]:
#checking the correlation

love_movies['DaysToValentine'].corr(love_movies['imdbRating'])


In [ ]:
"""
Low degree: When the value lies below +- 0.29, then it is said to be a small correlation.

As one variable increases, the other variable tends to also increase (if positive) or decrease (if negative).
"""

In [ ]:
#if days to valentines increases, then rating decreases, low degree correlation